# Mean-Field AOA

In [1]:
import numpy as np

In [2]:
def rotation(alpha: np.ndarray) -> np.ndarray:
    """Computes an array of rotation matrices parametrized by the vector alpha.

    To understand why the returned array is defined with an extra transpose, 
    consider the corresponding test in ../tests/test_MeanFieldQAOA.py!

    Args:
        alpha (np.ndarray, float):  vector of rotation angles or a scalar rotation angle

    Returns:
        np.ndarray: array of rotation matrices corresponding to the input angles
    """
    c, s = np.cos(alpha), np.sin(alpha)
    return np.array(((c, -s), (s, c))).T

In [3]:
# schedule
p = 10
τ = 0.5

γ = τ * (np.arange(1, p + 1) - 1/2) / p
β = τ * (1 - np.arange(1, p + 1) / p)
β[p-1] = τ / (4 * p)

In [4]:
# initial spins
N = 5
S = np.array([[1., 0., 0.] for _ in range(N - 1)]) # fix final spin (i.e. leave it out)

In [5]:
np.random.seed(11)
J = np.random.normal(0, 1, size=(N, N)) / np.sqrt(N)
J = np.triu(J, k=1)
J = J + J.transpose()

In [6]:
# magnetization
def m(S: np.ndarray, J: np.ndarray) -> np.ndarray:
    """Computes the vector arising from the product of the interaction matrix J
    with the vector formed by all z-components of the spins S.

    Args:
        S (np.ndarray): matrix holding the x-,y- and z-components of all spins
        J (np.ndarray): interaction matrix

    Returns:
        np.ndarray: vector holding the results
    """
    return J[N-1, :-1] + J[:N-1, :N-1] @ S[:, 2]  # np.triu(J, k=1) @ S[:, 2]

# evolution matrices
V_D = lambda k: rotation(2 * β[k] * np.ones(N-1))
V_P = lambda k, m: rotation(2 * γ[k] * m)

In [7]:
import time

In [8]:
# evolution
t_0 = time.time()
for k in range(p):
    XY = np.einsum("ijk,ik->ij", V_P(k, m(S, J)), S[:, :2])
    S = np.concatenate((XY[:, :1],
                        np.einsum("ijk,ik->ij", V_D(k),
                                  np.concatenate((XY[:, 1:], S[:, 2:]), axis=1))
                       ), axis=1)
    
t_f = time.time()
t_f - t_0

0.0013113021850585938

In [9]:
S_test = np.array([[-0.4280189887648497, -0.4161436735954462, -0.3151885316091266, 0.06825210700086091], 
                   [0.57845514021309, 0.1965348184395327, -0.14809317283731896, -0.5908423629410464], 
                   [0.6943985858408479, -0.8878054449413042, -0.9374031159010826, -0.8038948638001017]])

In [10]:
np.allclose(S, S_test.T)

True